In [139]:
from sklearn.cluster import KMeans
from sklearn import datasets
import numpy as np

# 5 clusters
# output is an array of ints where each int 
# is the cluster it belongs to in this 0-4

iris = datasets.load_iris()
x = iris["data"][:, (2,3)]
y = (iris["target"] == 2).astype(np.float64)
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(x)
print(y_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 3 2 2 2 3 2 3 3 2 3 2 3 2 2 3 2 3 2 3 2 2
 2 2 2 2 2 3 3 3 3 2 2 2 2 2 3 3 2 2 3 3 3 3 3 2 3 3 4 0 4 0 4 4 2 4 4 4 0
 0 0 0 0 0 0 4 4 2 0 0 4 2 0 4 2 2 0 4 4 4 0 2 0 4 0 0 2 0 0 0 0 4 0 0 0 0
 0 0]


In [ ]:
kmeans.cluster_centers_

In [ ]:
x_new = np.array([[0, 2], [3,2], [-3, 3], [-3, 2.5]])
kmeans.predict(x_new)

In [ ]:
# assigning each data point to a cluster is hard clustering
# we can also give each point a score this is called soft clustering

# transform measures distance from the point to each clusters mean
kmeans.transform(x_new)

In [ ]:
# if you have a rough idea of where centroids should be just set run it with that and n_init as 1

good_init = np.array([[-3,3], [-3, 2]])
kmeans = KMeans(n_clusters=2, init=good_init, n_init=1)
x_new = np.array([[0, 2], [3,2], [-3, 3], [-3, 2.5]])
kmeans.fit_predict(x_new)
kmeans.predict(x_new)
# the best way tho is to init randomly and pick the best one, n_init is 10 by default
# the best one is the one with the lowest inertia, which is the mse between each instance and it's centroid
kmeans.inertia_
# return negative inertia with score, it will try to maximize the score
kmeans.score(x)

In [141]:
# minibatch kmeans runs the algorithm with batches
# if the dataset still doesn't fit in memory go back to dimensionality reduction and use memmap
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(x)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=5, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [ ]:
# silhouette score
# (b-a) / max(a,b)
# a is the mean distance to other points in the same cluster
# b is mean distance to the instances of the nearest cluster
# scale of -1 to +1 
# +1 means it's in the middle of it's cluster and far from the middle of the other
# -1 means the opposite and that it has probably been assigned to the wrong one
# plot each one over a range of k to get a silhouette diagram

from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(x)

silhouette_score(x, kmeans.labels_)

In [ ]:
# You can compare the silhouette scores for different numbers of clusters with a silhouette diagram
# The best choice is the one where every cluster has a silhouette score above the mean, meaning all its clusters are
# distinct from each other

import matplotlib
from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter

plt.figure(figsize=(11, 9))


kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(x.reshape(-1, 1))
                for k in range(1, 10)]

silhouette_scores = [silhouette_score(x.reshape(-1, 1), model.labels_)
                     for model in kmeans_per_k[1:]]

for k in (3, 4, 5, 6):
    plt.subplot(2, 2, k - 2)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(x.reshape(-1, 1), y_pred)

    padding = len(x) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()

        color = matplotlib.cm.Spectral(i / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    if k in (3, 5):
        plt.ylabel("Cluster")
    
    if k in (5, 6):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--")
    plt.title("$k={}$".format(k), fontsize=16)

plt.show()

In [ ]:
# example code for image segmentation based on color
# breaking the image into segments, assigning each pixel to a segment
# ex: self driving cars use this for objectx detection

# from matplotlib.image import imread
# import os

# image = imread(os.path.join("images", "unsupervised_learning", "ladybug.png"))
# image.shape

# x = image.reshape(-1, 3)
# kmeans = KMeans(n_clusters=8).fit(x)
# segmented_img = kmeans.cluster_centers_[kmeans.labels_]
# segmented_img = segmented_img.reshape(image.shape)

In [ ]:
from sklearn.datasets import load_digits

x_digits, y_digits = load_digits(return_X_y=True)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_digits, y_digits)

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

log_reg.score(x_test, y_test)

In [ ]:
# you can use clustering for pre processing and dimensionality reduction
# using an arbitrary number of clusters here:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50)),
    ("log_reg", LogisticRegression())
])
pipeline.fit(x_train, y_train)
pipeline.score(x_test, y_test)

In [ ]:
# we can use gridsearch to find the best number of clusters to use in the preprocessing

from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(x_train, y_train)

print(grid_clf.best_params_)
print(grid_clf.score(x_test, y_test))

In [ ]:
# we can also use clustering for semi supervised learning

n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(x_train[:n_labeled], y_train[:n_labeled])

log_reg.score(x_test, y_test)

# use kmeans to find the representative images which are closest to the centroids
# we will use these to classify the other unlabeled images
k = 50
kmeans = KMeans(n_clusters=k)
x_digits_dist = kmeans.fit_transform(x_train)
representative_digit_idx = np.argmin(x_digits_dist, axis=0)
x_representative_digits = x_train[representative_digit_idx]

from matplotlib import pyplot as plt

plt.figure(figsize=(8, 2))
for index, x_representative_digit in enumerate(x_representative_digits):
    plt.subplot(k // 10, 10, index + 1)
    plt.imshow(x_representative_digit.reshape(8, 8), cmap="binary", interpolation="bilinear")
    plt.axis('off')
plt.show()

In [ ]:
# now we can manually label a section of images, and use that as the labels for our training data
y_representative_digits = np.array([ 
    3, 9, 2, 6, 7, 9, 0, 2, 1, 6,
    4, 9, 4, 2, 0, 5, 1, 4, 7, 5,
    3, 2, 7, 3, 5, 9, 6, 9, 1, 7, 
    2, 8, 8, 5, 4, 5, 7, 8, 3, 0,
    6, 1, 5, 2, 9, 8, 8, 8, 6, 1
])

# and use it to train a classifier
log_reg = LogisticRegression()
log_reg.fit(x_representative_digits, y_representative_digits)
log_reg.score(x_test, y_test)

In [ ]:
# we can improve on this by propagating the labels to all the instances in the same cluster

y_train_propagated = np.empty(len(x_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]
    
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train_propagated)
log_reg.score(x_test, y_test)


In [ ]:
# a better idea is to propagate to only the labels closest to the centroids, not the ones on the boundaries

percentile_closest = 20

x_cluster_dist = x_digits_dist[np.arange(len(x_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = x_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (x_cluster_dist > cutoff_distance)
    x_cluster_dist[in_cluster & above_cutoff] = -1
partially_propagated = (x_cluster_dist != -1)
x_train_partially_propagated = x_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(x_train_partially_propagated, y_train_partially_propagated)
log_reg.score(x_test, y_test)

In [ ]:
# DBSCAN algorithm
# for each instance it counts how many instances located a certain distance from it
# If an instance has at least min_samples from it it is a core instance
# All instances in the same neighborhood of a core instance belong to its cluster
# Anything that isn't a core instance or isn't a neighborhood of one is an anomaly

from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

x, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(x)

# the ones that have -1 don't any cluster and are anomalies
dbscan.labels_

In [ ]:
# dbscan doesn't have a predict method for new instances only fit_predict
# if you want to predict new instances u need to implement something else
# we can use knn on new instances and add them to our dbscan clusters

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

x_new = np.array([[-0.5, 0],[0, 0.5], [1,-0.1], [2,1]])
print(knn.predict(x_new))
print(knn.predict_proba(x_new))

In [ ]:
# you can make this KNeighbors classifier account for outliers with a max distance field

y_dist, y_pred_idx = knn.kneighbors(x_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist > 0.2] = -1

y_pred.ravel()

In [ ]:
# Gaussian mixtures, clusters formed from a distribution based on the input data
# given a dataset X first start by estimating the weights and distribution parameters
# GM is like a KM but it also finds the size, shape and orientation of the clusters, not just the means

from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(x)

print(gm.weights_)
print(gm.means_)
print(gm.covariances_)
print(gm.converged_)
print(gm.n_iter_)
print(gm.predict(x))
print(gm.predict_proba(x))

In [ ]:
# since GM's generate a distribution you can get new instances by sampling them
x_new, y_new = gm.sample(6)
print(x_new)
print(y_new)

In [ ]:
# you can estimate the density of the function at any point, aka how dense the cluster will be

print(gm.score_samples(x))

In [ ]:
# you can constrain it's shape by setting the covariance_type hyperparameter
#gm = GaussianMixture(n_components=3, n_init=10, covariance_type="spherical") must be spheres
#gm = GaussianMixture(n_components=3, n_init=10, covariance_type="diag") ellipsoid must be parallel to coordinate axes
#gm = GaussianMixture(n_components=3, n_init=10, covariance_type="tied") all clusters must be the same

In [ ]:
# gm's are the best for anomaly detection
# identifying outliers with a 4% defective threshold

densities = gm.score_samples(x)
density_threshold = np.percentile(densities, 4)
anomalies = x[densities < density_threshold]
print(anomalies)

In [ ]:
# you can't find the number of clusters with inertia or silhouette
# you have use either Bayesian information criterion BIC = log(m)p - 2log(L) 
# or Akaike information criterion AIC = 2p - 2log(L)
# m = # of instances
# p = # of parameters being learned by the model
# L = maximized likelihood

print(gm.bic(x))
print(gm.aic(x))

In [ ]:
# bayesian gaussian mixture helps select the number of clusters
# set n_clusters higher than you think is necessary and automatically gets rid of the unnecessary ones
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(x)
print(np.round(bgm.weights_, 2))

In [169]:
# Return the anomaly score of each sample using the IsolationForest algorithm

# The IsolationForest ‘isolates’ observations by randomly selecting a feature
# and then randomly selecting a split value between the maximum and minimum values
# of the selected feature. Since recursive partitioning can be represented
# by a tree structure, the number of splittings required to isolate a sample
# is equivalent to the path length from to the path length from the root node
# to the terminating node. This path length, averaged over a forest of such random trees 
# is a measure of normality and our decision function. Random partitioning produces
# noticeably shorter paths for anomalies. Hence, when a forest of random trees
# collectively produce shorter path lengths for particular samples, 
# they are highly likely to be anomalies.

# Essentially it splits the space up with random bisection trees
# until it can completely isolate the instance, if it is easier to isolate 
# (lower node depth in the tree) then it is likely to be an anomaly

from sklearn.ensemble import IsolationForest

iForest = IsolationForest(n_estimators=100,  contamination=0.1)
iForest.fit_predict(x)
# -1 are classified as anomalies, can change sensitivity by changing contamination parameter

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1,
        1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1])

In [164]:
# The anomaly score of each sample is called Local Outlier Factor. 
# It measures the local deviation of density of a given sample with respect to its neighbors. 
# It is local in that the anomaly score depends on how isolated the object is with respect 
# to the surrounding neighborhood. More precisely, locality is given by k-nearest neighbors, 
# whose distance is used to estimate the local density. By comparing the local density of a sample 
# to the local densities of its neighbors, one can identify samples that have a substantially 
# lower density than their neighbors. These are considered outliers.

from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=2)
print(clf.fit_predict(x))

[ 1  1  1  1  1  1  1  1  1 -1  1  1 -1  1 -1  1  1  1  1 -1 -1  1  1  1
  1  1 -1  1  1  1  1  1 -1  1  1 -1  1 -1  1  1 -1 -1  1  1  1  1  1  1
  1  1  1  1  1  1 -1  1  1  1  1 -1  1 -1  1  1 -1  1  1  1  1  1  1  1
  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1  1  1 -1  1  1  1
  1  1 -1 -1  1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1 -1  1  1  1 -1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1]


In [182]:
# If you want to use it for novelty detection 
# (checking if new data is an outlier after fitting)
# you must set novelty = True
clf = LocalOutlierFactor(n_neighbors=2, novelty=True, contamination=0.15)
clf.fit(x)
x_new = np.array([[-0.5, 0],[0, 0.5], [1,-0.1], [2,1]])
# this shows the normality of the data, inliers here ~-1, outliers will be < -1
print(clf.negative_outlier_factor_)
clf.predict(x_new)

[-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00592232e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -5.00000001e+08 -1.00000000e+00 -1.00000000e+00
 -5.00000001e+08 -1.41421356e+00 -5.00000001e+08 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -9.14213562e-01 -1.00000000e+09
 -1.00000000e+09 -1.00000000e+00 -1.45710678e+00 -1.06066017e+00
 -1.50000000e+00 -1.00000000e+00 -5.00000001e+08 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -5.00000001e+08 -1.00000000e+00 -1.00000000e+00 -5.00000001e+08
 -1.00000000e+00 -5.00000001e+08 -1.00000000e+00 -1.00000000e+00
 -5.00000001e+08 -5.00000001e+08 -1.00000000e+00 -1.42258898e+00
 -1.32842712e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+09 -1.00000000e+00
 -1.10355339e+00 -1.00000000e+00 -1.00000000e+00 -1.41421356e+09
 -1.00000000e+00 -2.00000

array([-1, -1,  1, -1])